# Pipelines
Le but de ce notebook est de créer diverses pipelines allant de la préparation des données à l'entraînement du modèle et de les comparer afin de déterminer la stratégie permettant d'obtenir le meilleur score sur l'ensemble de validation.

In [1]:
from utils import load_data, train_valid_test_split, PipelineEvaluator
import estimators as est
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

In [2]:
# Chargement des données
df = load_data("./data/train.txt")

# Découpage des données
df_train, df_valid, df_test = train_valid_test_split(df, size=(600, 200, 100))

X = df["text"]
X_train = df_train["text"]
y_train = df_train["language"]
X_valid = df_valid["text"]
y_valid = df_valid["language"]
X_test = df_test["text"]
y_test = df_test["language"]

# Évaluation des pipelines
peval = PipelineEvaluator(X_train, y_train, X_valid, y_valid)

# Baseline

In [3]:
pipe = make_pipeline(CountVectorizer(), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

!!! BEST !!!


0.6654162399577045

# Caractères
La correction des caractères spéciaux ne change pratiquement pas le score.

In [4]:
pipe = make_pipeline(est.CorrectSpecialChars(), CountVectorizer(), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

!!! BEST !!!


0.6665963286741187

# Nombres
La tokenization des nombres n'a pratiquement aucun effet sur le score.

In [5]:
pipe = make_pipeline(est.TokenizeNumbers(), CountVectorizer(), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

!!! BEST !!!


0.6666167551065664

# CountVectorizer
On cherche à optimiser les paramètres de CountVectorizer.

In [6]:
pipe = make_pipeline(CountVectorizer(lowercase=False), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

!!! BEST !!!


0.6750873709348387

Ne pas passer en lowercase le texte permet de gagner environ 1 point de f1 score.

In [7]:
pipe = make_pipeline(CountVectorizer(stop_words="english"), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

0.6098206087166702

On a la preuve qu'il ne faut surtout pas enlever les stop words car cela fait perdre 5 points de f1 score.

In [8]:
pipe = make_pipeline(CountVectorizer(ngram_range=(1, 2)), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

!!! BEST !!!


0.7284038038419024

In [9]:
pipe = make_pipeline(CountVectorizer(lowercase=False, ngram_range=(1, 2)), LinearSVC(random_state=42, max_iter=1500))
peval.evaluate(pipe)

!!! BEST !!!


0.7381868073768907

Utiliser des unigrammes et des bigrammes permet d'augmenter nettement le f1 score de 8 points par rapport à la baseline.